In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")
print(os.environ["LANGCHAIN_TRACING_V2"])

[API KEY]
sk-proj-2ALiWBzcJl4s9ri6EUJ6T3BlbkFJxbUDIanlzbIf6JMPE7o2
true


In [3]:
import torch

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts.chat import PromptTemplate, ChatPromptTemplate

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from transformers import AutoModel, AutoTokenizer

from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager


# EEVE 모델 사용
llm = ChatOllama(
    temperature=0.7,
    top_k=40,
    top_p=0.9,
    model="EEVE:latest",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

/home/choi/anaconda3/envs/RAG_langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
current_directory = os.getcwd()
print(current_directory)

/home/choi/Git/RAG_con_doc/langchain


In [5]:
# 1단계 : 문서 로드

loader = TextLoader("./data/1Way_slab_design.txt")
docs = loader.load()
print(f"문서의 페이지수:{len(docs)}")




문서의 페이지수:1


In [6]:
# 2단계 : 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의 수 : {len(split_documents)}")

분할된 청크의 수 : 7


In [11]:
# 3단계 : 임베딩 (결정)
# ko-sroberta-multitask 임베딩 사용
import torch
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name = 'jhgan/ko-sroberta-multitask')


In [12]:
# 단계 4: DB 생성 및 저장
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [13]:
# 단계 5: 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()

In [20]:
# 검색기 테스트
retriever.invoke("슬래브의 콘크리트 전단강도는 어떻게 계산돼?")

[Document(page_content='$$\nV_{c}=\\frac{1}{6} \\sqrt{f_{c k}} b d\n$$\n\n\\begin{enumerate}\n  \\setcounter{enumi}{2}\n  \\item 일방향 슬래브에서 처짐을 계산하지 않는 경우의 최소두께 계산은 다음과 같습니다. 사용자가 입력한 슬래브 두께가 다음 최소 두께보다 작을 경우는 유효강성에 의한 첨짐량을 계산합니다.\n\\end{enumerate}\n\n\\begin{center}\n\\begin{tabular}{|c|c|}\n\\hline\n지지조건 & 최소두께 \\\\\n\\hline\n양단 단순지지조건 & $l / 20$ \\\\\n\\hline\n양단 연속지지조건 & $l / 28$ \\\\\n\\hline\n일단 연속 + 타단 힌지조건 & $l / 24$ \\\\\n\\hline\n캔틸레버 & $l / 10$ \\\\\n\\hline\n\\end{tabular}\n\\end{center}\n\n이 표의 값은 보통콘크리트 $\\left(\\omega_{c}=2,300 \\mathrm{~kg} / \\mathrm{m}^{3}\\right)$ 와 설계기준항복강도 $400 \\mathrm{~N} / \\mathrm{mm}^{2}$ 철근을\n\n사용한 부재에 대한 값이며 다른 조건에 대해서는 그 값을 다음과 같이 수정하여야 합니다.\n\n(1) $1,500 \\sim 2,000 \\mathrm{~kg} / \\mathrm{m}^{3}$ 범위의 단위질량을 갖는 구조용 경량콘크리트에 대해서는 계산된 $h$ 값 에 $\\left(1.65-0.00031 \\omega_{c}\\right)$ 를 곱해야 하지만 1.09 보다 작지 않아야 합니다.\n\n(2) $F_{y}$ 가 $400 \\mathrm{~N} / \\mathrm{mm}^{2}$ 이외인 경우는 계산된 $h$ 값에 $\\left(0.43+f_{y} / 700\\right)$ 를 곱하여야 합니다.\n\n\\begin{e

In [15]:
# 단계 6: 프롬프트 생성(Create Prompt)
prompt = ChatPromptTemplate.from_template(
    '''You are a helpful assistant, conversing with a user about the subjects contained in a set of documents.
Use the information from the DOCUMENTS section to provide accurate answers. If unsure or if the answer
isn't found in the DOCUMENTS section, simply state that you don't know the answer. Please answer me in korean. 
#Question:
{question}
#Context:
{context}
'''
)

In [16]:

# 단계 7: LLM 생성
# llm = ChatOllama(model='EEVE:latest')
# llm = ChatOpenAI(model_name='gpt-4o')
# llm = ChatOllama(model='llama3_kor:latest')

In [17]:
# 단계 8: 체인 생성
chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [24]:
# 질문 실행

question = "f_{c k} 가 20 b가 5 d 가 10일때 V_{c}는 몇이야?"

response = chain.invoke(question)
print(response)

여기 1방향 슬래브 설계를 위한 지침에 대한 번역본이 있습니다:

슬래브 설계의 시작점, 전단강도 및 최소두께 계산:

1. 연속 지지조건
   전단강도는 다음과 같이 계산합니다:
   $$V_{c}=\frac{1}{6}\sqrt{f_{c}k}bd$$
   여기서 $f_{ck}$는 보통콘크리트 단위 면적당 계수하중, $b$는 슬래브 폭, $d$는 슬래브 두께입니다.
2. 처짐을 계산하지 않는 최소두께:
   사용자가 입력한 슬래브 두께가 다음 값보다 작으면 유효강성에 의한 첨짐량을 고려합니다.
   | 지지조건 | 최소두께 |
   |-----------|----------|
   | 양단 단순지지조건 | $l / 20$ |
   | 양단 연속지지조건 | $l / 28$ |
   | 일단은 연속 + 타단은 힌지 | $l / 24$ |
   | 캔틸레버 | $l / 10$ |
   여기서 $l$은 경간입니다.
3. 수정사항:
   - 구조용 경량콘크리트의 경우, 계산된 $h$ 값에 $(1.65-0.00031 \omega_{c})$를 곱하고 1.09보다 작지 않도록 합니다.
   - 설계기준 항복강도 $400 N/mm^2$ 이외의 철근은 계산된 $h$ 값에 $(0.43+\frac{f_y}{700})$를 곱합니다.
4. 최소철근비:
   - $f_{y} \leq 400 N/mm^2$ 인 경우 0.0020
   - $f_{y}>400 N/mm^2$ 일 때는 $\frac{0.0020}{f_y}$입니다.
5. 철근 중심간격:
   - 건조환경에서의 최소값은 $375\left(\frac{280}{f_{s}}\right)-2.5 c_{c}$와 $300\left(\frac{280}{f_{s}}\right)$이며, $f_s$는 $f_y$의 2/3입니다.
   - 기타 환경에서의 최소값은 $375\left(\frac{210}{f_{s}}\right)-2.5 c_{c}$와 $300\left(\frac{210}{f_{s}}\right)$이며, $f_s$는 $f_y$의 2/3입니다.
